### Load libraries

In [2]:

from scipy.spatial.distance import jensenshannon
import numpy as np
import sys

sys.path.insert(0, '../src/')

from load_modify_sample_utils import load_model, get_objective_functions, get_reaction_bounds, modify_model
from load_modify_sample_utils import sample_dingo, sample_optgp

from correlations_utils import compute_copula
from correlations_utils import plot_copula


/home/touliopoulos/.local/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/touliopoulos/.local/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (
/home/touliopoulos/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


### Load model

In [3]:

ec_cobra_model, ec_cobra_reactions, ec_dingo_model, ec_dingo_reactions = load_model("../ext_data/models/e_coli_core.xml")

objective_functions = get_objective_functions(ec_cobra_model)
print(objective_functions)

default_reaction_bounds = get_reaction_bounds(ec_cobra_model)
print(default_reaction_bounds)


Set parameter Username
Set parameter LicenseID to value 2642044
Academic license - for non-commercial use only - expires 2026-03-25
['BIOMASS_Ecoli_core_w_GAM']
{'PFK': (0.0, 1000.0), 'PFL': (0.0, 1000.0), 'PGI': (-1000.0, 1000.0), 'PGK': (-1000.0, 1000.0), 'PGL': (0.0, 1000.0), 'ACALD': (-1000.0, 1000.0), 'AKGt2r': (-1000.0, 1000.0), 'PGM': (-1000.0, 1000.0), 'PIt2r': (-1000.0, 1000.0), 'ALCD2x': (-1000.0, 1000.0), 'ACALDt': (-1000.0, 1000.0), 'ACKr': (-1000.0, 1000.0), 'PPC': (0.0, 1000.0), 'ACONTa': (-1000.0, 1000.0), 'ACONTb': (-1000.0, 1000.0), 'ATPM': (8.39, 1000.0), 'PPCK': (0.0, 1000.0), 'ACt2r': (-1000.0, 1000.0), 'PPS': (0.0, 1000.0), 'ADK1': (-1000.0, 1000.0), 'AKGDH': (0.0, 1000.0), 'ATPS4r': (-1000.0, 1000.0), 'PTAr': (-1000.0, 1000.0), 'PYK': (0.0, 1000.0), 'BIOMASS_Ecoli_core_w_GAM': (0.0, 1000.0), 'PYRt2': (-1000.0, 1000.0), 'CO2t': (-1000.0, 1000.0), 'RPE': (-1000.0, 1000.0), 'CS': (0.0, 1000.0), 'RPI': (-1000.0, 1000.0), 'SUCCt2_2': (0.0, 1000.0), 'CYTBD': (0.0, 1000.

### Modify model and perform sampling

In [4]:

ec_cobra_model_condition_100, ec_dingo_model_condition_100 = modify_model(ec_cobra_model, objective_function="BIOMASS_Ecoli_core_w_GAM", optimal_percentage=100)
ec_cobra_model_condition_0, ec_dingo_model_condition_0 = modify_model(ec_cobra_model, objective_function="BIOMASS_Ecoli_core_w_GAM", optimal_percentage=0)


samples_dingo_condition_100 = sample_dingo(ec_dingo_model_condition_100, reaction_in_rows = True, ess=2000)
print(samples_dingo_condition_100.shape)

samples_dingo_condition_0 = sample_dingo(ec_dingo_model_condition_0, reaction_in_rows = True, ess=2000)
print(samples_dingo_condition_0.shape)


Read LP format model from file /tmp/tmpzb2b1j3p.lp
Reading time = 0.01 seconds


: 72 rows, 190 columns, 720 nonzeros
Read LP format model from file /tmp/tmp0zqfqo_y.lp
Reading time = 0.01 seconds
: 72 rows, 190 columns, 720 nonzeros
Set parameter Username
Set parameter LicenseID to value 2642044
Academic license - for non-commercial use only - expires 2026-03-25
phase 1: number of correlated samples = 500, effective sample size = 11, ratio of the maximum singilar value over the minimum singular value = 1091.97
phase 2: number of correlated samples = 500, effective sample size = 20, ratio of the maximum singilar value over the minimum singular value = 84.0129
phase 3: number of correlated samples = 500, effective sample size = 4, ratio of the maximum singilar value over the minimum singular value = 279.746
phase 4: number of correlated samples = 500, effective sample size = 27, ratio of the maximum singilar value over the minimum singular value = 89.9354
phase 5: number of correlated samples = 500, effective sample size = 192, ratio of the maximum singilar value ov

[5]maximum marginal PSRF: 1.01166


phase 1: number of correlated samples = 500, effective sample size = 4, ratio of the maximum singilar value over the minimum singular value = 1131
phase 2: number of correlated samples = 500, effective sample size = 156, ratio of the maximum singilar value over the minimum singular value = 1
phase 3: number of correlated samples = 2400, effective sample size = 941
phase 4: number of correlated samples = 2400, effective sample size = 855
phase 5: number of correlated samples = 300, effective sample size = 74
[5]total ess 2030: number of correlated samples = 6100


(95, 6100)


[5]maximum marginal PSRF: 1.00267


Below a test function shows how the corresponding `copula_tail_dependence` works in `correlations_utils.py`.

We can see which are the cells (positions) that are selected in each edge of the copula.

In [21]:

#import plotly.io as pio
#pio.renderers.default = 'browser'

def copula_tail_dependence_test(copula, cop_coeff_1, cop_coeff_2, cop_coeff_3):

    rows, cols = copula.shape
    
    red_mass = 0
    blue_mass = 0
        
    top_left_count = 0
    bottom_right_count = 0
    top_right_count = 0
    bottom_left_count = 0
    
    top_left = []
    bottom_right = []
    top_right = []
    bottom_left = []
    
                    
    for row in range(rows):
        for col in range(cols):
            # values in the diagonal
            if ((row-col >= -cop_coeff_1*rows) & (row-col <= cop_coeff_1*rows)): 
                # values near the top left
                if (row+col < cop_coeff_2*rows -1):
                    red_mass = red_mass + copula[row][col]       
                    top_left_count = top_left_count + 1
                    top_left.append((row, col))
                    
                # values near the bottom right
                elif (row+col > cop_coeff_3*rows -1):
                    red_mass = red_mass + copula[row][col]       
                    bottom_right_count = bottom_right_count + 1
                    bottom_right.append((row, col))
            
            # values in the other diagonal
            else:
                # values near the top right and bottom left corner
                if (row+col >= cop_coeff_2*rows -1) & (row+col <= cop_coeff_3*rows -1):                    
                    # values near the top right
                    if row < rows / 2:
                        blue_mass = blue_mass + copula[row][col]       
                        top_right_count = top_right_count + 1
                        top_right.append((row, col))
                    
                    # values near the bottom left
                    elif row >= rows / 2:
                        blue_mass = blue_mass + copula[row][col]       
                        bottom_left_count = bottom_left_count + 1
                        bottom_left.append((row, col))
                                                

    #print(top_left, "\n", bottom_right, "\n", top_right, "\n" , bottom_left)
    #print(top_left_count, bottom_right_count, top_right_count, bottom_left_count)
    print(red_mass, blue_mass, red_mass/blue_mass)
    

In [23]:

n1 = ec_dingo_reactions.index("PGK")
n2 = ec_dingo_reactions.index("PFK")

flux1 = samples_dingo_condition_0[n1]
flux2 = samples_dingo_condition_0[n2]

data_flux1=[flux1, ec_dingo_reactions[n1]]
data_flux2=[flux2, ec_dingo_reactions[n2]]


In [24]:

cells = 10
plot_copula(data_flux1, data_flux2, n = cells)


In [ ]:

# parameters for the width of the copula's diagonal
cop_coeff = 0.2
cop_coeff_1 = cop_coeff
cop_coeff_2 = 1 - cop_coeff
cop_coeff_3 = 1 + cop_coeff

copula = compute_copula(flux1, flux2, n = cells)
copula_tail_dependence_test(copula, cop_coeff_1, cop_coeff_2, cop_coeff_3)


0.2504918032786885 0.40426229508196726 0.619626926196269


In [25]:

copula_flat = copula.flatten()

reference_copula = np.full( (cells, cells), (1 / (cells*cells)) )
reference_copula_1_flat = reference_copula.flatten()


np.set_printoptions(threshold=np.inf)
print(copula_flat)
print("\n")
print(reference_copula)
print("\n")


dist = jensenshannon(copula_flat, reference_copula_1_flat)
print(dist)


[0.         0.         0.0095082  0.01163934 0.01409836 0.01262295
 0.01196721 0.01131148 0.01360656 0.0152459  0.         0.00262295
 0.01016393 0.01295082 0.01278689 0.01131148 0.01131148 0.01409836
 0.0104918  0.0142623  0.         0.00819672 0.00967213 0.01442623
 0.01163934 0.01180328 0.01213115 0.01065574 0.01114754 0.01032787
 0.         0.01180328 0.01131148 0.01163934 0.01098361 0.00868852
 0.00918033 0.01196721 0.01327869 0.01114754 0.00081967 0.01377049
 0.01032787 0.0095082  0.00983607 0.01081967 0.01147541 0.01196721
 0.01131148 0.01016393 0.0057377  0.01704918 0.01114754 0.00901639
 0.00868852 0.01081967 0.00983607 0.01016393 0.00885246 0.00868852
 0.00983607 0.01131148 0.01163934 0.00836066 0.00868852 0.00967213
 0.00983607 0.00885246 0.00983607 0.01196721 0.01901639 0.01360656
 0.00983607 0.00737705 0.00836066 0.00918033 0.00983607 0.00754098
 0.00836066 0.00688525 0.02606557 0.01       0.00868852 0.00819672
 0.01       0.00852459 0.00786885 0.00737705 0.00672131 0.0065

### Below arrays representing copulas are created to show how the jensenshannon distance works

Note: As long as you do not describe what we are looking at, it's hard to follow. Please describe the main conclusions from the jensenshannon experiments, as well as of the copulas above.

It's not clear what you want to show in each case and/or why you are doing what you're doing..

In [ ]:


cells = 5
reference_copula = np.full( (cells, cells), (1 / (cells*cells)) )

reference_copula_1_flat = reference_copula.flatten()



reference_copula_2 = np.array([[0.25, 0.25, 0, 0, 0],
                               [0.25, 0.25, 0, 0, 0],
                               [0, 0, 0,  0, 0],
                               [0, 0, 0,  0, 0],
                               [0, 0, 0, 0,  0]])

reference_copula_2_flat = reference_copula_2.flatten()



reference_copula_3 = np.array([[1, 0, 0, 0, 0],
                               [0, 0, 0, 0, 0],
                               [0, 0, 0,  0, 0],
                               [0, 0, 0,  0, 0],
                               [0, 0, 0, 0,  0]])

reference_copula_3_flat = reference_copula_3.flatten()



0.0
0.6785698641197397
0.7799839057460577


In [10]:

# Compute Jensen-Shannon distance (range: 0 to 1)
dist = jensenshannon(reference_copula_1_flat, reference_copula_1_flat)
print(dist)

dist = jensenshannon(reference_copula_1_flat, reference_copula_2_flat)
print(dist)

dist = jensenshannon(reference_copula_1_flat, reference_copula_3_flat)
print(dist)


0.0
0.6785698641197397
0.7799839057460577


In [ ]:

cells = 100

a = np.zeros((cells,cells))

b = a.copy()
b[0,0] = 1
b_flat = b.flatten()

c = a.copy()
c[:10, :10] = 0.01
c_flat = c.flatten()

reference_copula = np.full( (cells, cells), (1 / (cells*cells)) )
reference_copula_flat = reference_copula.flatten()


In [ ]:

dist = jensenshannon(b_flat, reference_copula_flat)
print(dist)

dist = jensenshannon(c_flat, reference_copula_flat)
print(dist)

0.8322479564657576
0.8155344336992489
